# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [51]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct
from pyspark.sql.types import ShortType

spark = SparkSession.builder.appName('lr_proj').getOrCreate()

In [4]:
data = spark.read.csv("cruise_ship_info.csv", inferSchema=True, header=True)

In [6]:
data.show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 4 rows



In [31]:
distinct_cruise_lines = data.select(countDistinct('Cruise_line')).collect()[0][0]

In [53]:
indexer_str = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_num')
df_indexed = indexer_str.fit(data).transform(data)
df_indexed.show(5)
df_indexed.printSchema()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_num|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+

In [54]:
indexer_line = OneHotEncoder(inputCol="Cruise_line_num", outputCol="Cruise_line_index")
df_indexed = indexer_line.transform(df_indexed)

df_indexed.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_num: double (nullable = false)
 |-- Cruise_line_index: vector (nullable = true)



In [61]:
assembler = VectorAssembler(
              inputCols = ('Age', 'Tonnage', 'passengers', 'length', 
                           'cabins', 'passenger_density', 'Cruise_line_index'),
              outputCol='features')
data_for_algo = assembler.transform(df_indexed)
data_for_algo = data_for_algo.select('features', 'Crew')

### Train model

In [65]:
train_data,test_data = data_for_algo.randomSplit([0.8,0.2])

In [66]:
lr = LinearRegression(labelCol='Crew')

In [67]:
lrModel = lr.fit(train_data)

In [68]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [0.005844605294732867,0.005569236944163535,-0.10173100261854323,0.5517028988497154,0.7752399729458767,0.001084588031373102,-1.1628639841798485,0.35069890176443097,0.036072855820690926,-0.5525478176776359,0.4219103912248934,-0.4267165651086184,0.6136276410742794,-0.12781378480566327,1.340020382251088,-0.05141372074517705,0.713507742915622,0.3094452386440124,0.6184802394454518,0.476349632261833,-0.21636842884243337,-0.4362420365265579,0.1322315155123019,0.6804482690047798,0.28496567441521115] Intercept: -2.1553685983925845


In [69]:
test_results = lrModel.evaluate(test_data)

In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))